## DSPY Integration for enhanced LLM prompting

In [1]:
!pip install dspy-ai

In [2]:
import dspy 

class BetterSentence(dspy.Signature):
    """Compare the two sentences and return the one which is more grammatically correct"""
    sentence1 = dspy.InputField()
    sentence2 = dspy.InputField()
    best_sentence = dspy.OutputField()

c:\Users\krish\miniconda3\envs\genAIProjects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os 

os.environ["GOOGLE_API_KEY"] = "<Your_GOOGLE_API_KEY>"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
# model = ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', temperature=0.4)
gemini = dspy.Google(model='gemini-1.5-pro-latest', api_key=os.environ["GOOGLE_API_KEY"], temperature=0.5)

In [5]:
dspy.settings.configure(lm=gemini)

from dspy.signatures.signature import signature_to_template

better_sentence_as_template = signature_to_template(BetterSentence)

In [6]:
print(better_sentence_as_template)

Template(Compare the two sentences and return the one which is more grammatically correct, ['Sentence 1:', 'Sentence 2:', 'Best Sentence:'])


In [7]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.program = dspy.ChainOfThought(BetterSentence)

    def forward(self, sentence1, sentence2):
        return self.program(sentence1=sentence1, sentence2=sentence2)

In [8]:
c = CoT()
a = c.forward(sentence1='I reading books and curating better articles for my readers.', sentence2='I enjoy reading books and curating better articles for my newletter readers.')

In [9]:
a

Prediction(
    rationale='Sentence 1: I reading books and curating better articles for my readers.\n\nSentence 2: I enjoy reading books and curating better articles for my newsletter readers.\n\nReasoning: Let\'s think step by step in order to produce the best sentence. We need to make sure the sentence has a subject, verb, and object and that they all agree. Sentence 1 is missing a verb.  Sentence 2 includes a verb, "enjoy" and also clarifies who the readers are.',
    best_sentence='I enjoy reading books and curating better articles for my newsletter readers.'
)

In [12]:
print(a.best_sentence)

I enjoy reading books and curating better articles for my newsletter readers.
